In [0]:
from random import randrange
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

batch_size=200
criterion = nn.NLLLoss()
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)
use_cuda = True

def shuffle(rndx,rndy,image):
    y = np.zeros((56,56))
    image = image.view(-1, 28)
    y[rndx:28+rndx,rndy:28+rndy]+=np.array(image)
    return torch.as_tensor(y).float()

def create_nn(learning_rate=0.01, epochs=10,
              log_interval=10):
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.layer1 = nn.Sequential(
                nn.Conv2d(1, 15, kernel_size=28, stride=1, padding=0),
                nn.ReLU()
                # ,nn.MaxPool2d(kernel_size=2, stride=2)
                )
            self.layer2 = nn.Sequential(
                nn.Conv2d(15, 32, kernel_size=4, stride=1, padding=3),
                nn.ReLU()
                ,nn.MaxPool2d(kernel_size=2, stride=2)
                )
            self.drop_out = nn.Dropout()
 
            #self.fc1 = nn.Linear(28 * 28 *4, 200)
            self.fc1 = nn.Linear(8192, 200)
            self.fc2 = nn.Linear(200, 200)
            self.f7 = nn.Linear(200, 10)

        def forward(self, x):
            x = self.layer1(x)
            x = self.layer2(x)
            x = x.reshape(x.size(0), -1)
            x = self.drop_out(x)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.f7(x)
            return F.log_softmax(x)

    net = Net()
    if use_cuda and torch.cuda.is_available():
        net.cuda()
    print(net)

    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

    # run the main training loop
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
            temp = torch.as_tensor(np.zeros((200, 1, 28*2, 28*2))).float()
            for i,(image) in enumerate(data):
                rndx = randrange(29)
                rndy = randrange(29)
                temp[i][0] = shuffle(rndx,rndy,image)
            data = temp
            if use_cuda and torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
            
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.data))
    return net

    # run a test loop
def test_nn():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)

        temp = torch.as_tensor(np.zeros((200, 1, 28*2, 28*2))).float()
        for i,(image) in enumerate(data):
            rndx = 0#randrange(29)
            rndy = 0#randrange(29)
            temp[i][0] = shuffle(rndx,rndy,image)
        data = temp

        if use_cuda and torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        net_out = net(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).data
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


if __name__ == "__main__":
    run_opt = 2
    if run_opt == 1:
        simple_gradient()
    elif run_opt == 2:
        net = create_nn()
        test_nn()

In [0]:
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)

        temp = torch.as_tensor(np.zeros((200, 1, 28*2, 28*2))).float()
        for i,(image) in enumerate(data):
            rndx = randrange(29)
            rndy = randrange(29)
            temp[i][0] = shuffle(rndx,rndy,image)
        data = temp

        if use_cuda and torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        net_out = net(data)
        # sum up batch loss
        test_loss += criterion(net_out, target).data
        pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')